### Load Pretrained Model
Load a pretrained Mamba Model that is compatible with Transformers Library

In [ ]:
from modeling_mamba import MambaForCausalLM
from transformers import AutoTokenizer

model = MambaForCausalLM.from_pretrained('Q-bert/Mamba-130M')
tokenizer = AutoTokenizer.from_pretrained('Q-bert/Mamba-130M')

text = "Hi"

input_ids = tokenizer.encode(text, return_tensors="pt")

output = model.generate(input_ids, max_length=20, num_beams=5, no_repeat_ngram_size=2)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

/Users/soma/opt/anaconda3/envs/shane/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/soma/opt/anaconda3/envs/shane/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Hi, I'm looking for a new job. I've been working at a company for about a


In [ ]:
print([(n, type(m)) for n, m in model.named_modules()])

[('', <class 'modeling_mamba.MambaForCausalLM'>), ('model', <class 'modeling_mamba.MambaModel'>), ('model.embedding', <class 'torch.nn.modules.sparse.Embedding'>), ('model.layers', <class 'torch.nn.modules.container.ModuleList'>), ('model.layers.0', <class 'modeling_mamba.MambaBlock'>), ('model.layers.0.in_proj', <class 'torch.nn.modules.linear.Linear'>), ('model.layers.0.conv1d', <class 'torch.nn.modules.conv.Conv1d'>), ('model.layers.0.x_proj', <class 'torch.nn.modules.linear.Linear'>), ('model.layers.0.dt_proj', <class 'torch.nn.modules.linear.Linear'>), ('model.layers.0.out_proj', <class 'torch.nn.modules.linear.Linear'>), ('model.layers.0.norm', <class 'modeling_mamba.MambaRMSNorm'>), ('model.layers.1', <class 'modeling_mamba.MambaBlock'>), ('model.layers.1.in_proj', <class 'torch.nn.modules.linear.Linear'>), ('model.layers.1.conv1d', <class 'torch.nn.modules.conv.Conv1d'>), ('model.layers.1.x_proj', <class 'torch.nn.modules.linear.Linear'>), ('model.layers.1.dt_proj', <class 'tor

In [ ]:
print(type(model))

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
    
print('plain',print_trainable_parameters(model))

<class 'modeling_mamba.MambaForCausalLM'>
trainable params: 129135360 || all params: 129135360 || trainable%: 100.0
plain None


### Add LoRA adapters
1. Identify a particular layer in the Mamba and add an LoRA layer there
2. At this time, is only layer to verify if the code works


In [ ]:
# adapter-1
from peft import LoraConfig, TaskType, get_peft_model

target_modules=["model.layers.3.x_proj"]

model = MambaForCausalLM.from_pretrained('Q-bert/Mamba-130M')

config = LoraConfig(
target_modules = target_modules,
task_type="CAUSAL_LM")
m1 = get_peft_model(model, config)
m1.print_trainable_parameters()
m1.save_pretrained("./wts/ada-1")

trainable params: 12,928 || all params: 129,148,288 || trainable%: 0.010010198509174199


In [ ]:
# adapter-2
from peft import LoraConfig, TaskType, get_peft_model

target_modules=["model.layers.2.x_proj"]
model = MambaForCausalLM.from_pretrained('Q-bert/Mamba-130M')

config = LoraConfig(
target_modules = target_modules,
task_type="CAUSAL_LM")
m2 = get_peft_model(model, config)
m2.print_trainable_parameters()
m2.save_pretrained("./wts/ada-2")

trainable params: 12,928 || all params: 129,148,288 || trainable%: 0.010010198509174199


### Merge the adpater into the Model
merge the adapter back to the model, so the merged model will have exactly the same architecture
except with the weights modified

In [ ]:
# https://stackoverflow.com/questions/77596271/i-want-to-merge-my-peft-adapter-model-with-the-base-model-and-make-a-fully-new-m

#from peft import PeftConfig, PeftModel
#base_model = MambaForCausalLM.from_pretrained('Q-bert/Mamba-130M')
#adapter_path = './wts/ada-1'
#m1 = PeftModel.from_pretrained(base_model, adapter_path)

In [ ]:
# https://github.com/huggingface/peft
from peft import PeftMixedModel


base_model = MambaForCausalLM.from_pretrained('Q-bert/Mamba-130M')
print('base mamba',print_trainable_parameters(base_model))
peft_model = PeftMixedModel.from_pretrained(base_model, adapter_path , "adapter1")
print('base mamba',print_trainable_parameters(base_model))
peft_model.load_adapter('./wts/ada-2', "adapter2")
peft_model.set_adapter(["adapter1", "adapter2"])
print('base mamba',print_trainable_parameters(base_model))

output = peft_model.generate(input_ids, max_length=20, num_beams=5, no_repeat_ngram_size=2)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

trainable params: 129135360 || all params: 129135360 || trainable%: 100.0
base mamba None
trainable params: 12928 || all params: 129148288 || trainable%: 0.010010198509174199
base mamba None
trainable params: 25856 || all params: 129161216 || trainable%: 0.02001839313745699
base mamba None
Hi, I'm looking for a new job. I've been working at a company for about a


In [ ]:
# createa combined fed model
peft_model.merge_and_unload()
peft_model.base_model.save_pretrained("./mbins/fed-lora/")